<a href="https://colab.research.google.com/github/VesleAnne/Barney_Stinson_Chatbot/blob/main/Notebooks/Prepairing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb langchain sentence-transformers pandas torch transformers nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 618.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.6 MB/s eta 

In [2]:
!pip install -U langchain-community -q
#!pip install chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
import chromadb
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModel
import nltk
import re
import tqdm
from nltk.tokenize import sent_tokenize

In [4]:
#import chardet

#def detect_encoding(file_path):
#    with open(file_path, 'rb') as file:
#        result = chardet.detect(file.read())
#        return result['encoding']

#encoding = detect_encoding(txt_path)
#print(f"Detected encoding: {encoding}")

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [8]:
current_directory = os.getcwd()
save_directory = os.path.join(current_directory, 'DATA')
os.makedirs(save_directory, exist_ok=True)

In [9]:
# Initialize ChromaDB client
client = chromadb.Client()

In [10]:
# Create separate collections for text, Excel text, and Excel context
text_collection = client.create_collection(name="text_embeddings")
csv_text_collection = client.create_collection(name="csv_text_embeddings")
csv_context_collection = client.create_collection(name="csv_context_embeddings")

In [11]:
# Initialize the multilingual model
model_name = "intfloat/multilingual-e5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [12]:
# Initialize LangChain's Hugging Face embeddings with the multilingual model
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [13]:
# Function to process text file (Bro code)
def chunk_text(text, chunk_size=2):
    """
    Chunk text into smaller segments of chunk_size sentences.
    """
    sentences = sent_tokenize(text)  # Ensure sentences are tokenized properly
    chunks = [' '.join(sentences[i:i + chunk_size]) for i in range(0, len(sentences), chunk_size)]
    return chunks

def process_text_file(txt_path):
    with open(txt_path, 'r', encoding='windows-1252') as file:
        text = file.read()

    # Chunk the text
    text_chunks = chunk_text(text)

    # Process and store each chunk
    for idx, chunk in enumerate(text_chunks):
        # Generate embedding
        embedding = hf_embeddings.embed_documents([chunk])  # Correct method name

        # Store in ChromaDB
        text_collection.add(
            ids=[f"chunk_{idx}"],  # Unique ID for each chunk
            embeddings=embedding,
            metadatas=[{"chunk_index": idx}],
            documents=[chunk]
        )

In [14]:
txt_filename = 'Stinson Barney. The Bro Code - BooksCafe.Net.txt'

In [15]:
txt_path = os.path.join(current_directory, 'DATA', txt_filename)
process_text_file(txt_path)

In [16]:
# Function to process csv file with phrases of the character
def process_csv_file(csv_path):
    # Load the CSV file into a DataFrame
    df_csv = pd.read_csv(csv_path)

    # Ensure 'text' and 'context' columns are treated as strings and handle NaN values
    df_csv['text'] = df_csv['text'].fillna('').astype(str)
    df_csv['context'] = df_csv['context'].fillna('').astype(str)

    for idx, row in df_csv.iterrows():
        text = row['text']
        context = row['context']

        # Generate and store embeddings for text
        if text:  # Check if text is not empty
            text_embedding = hf_embeddings.embed_documents([text])[0]  # Get the first (and only) embedding from the list
            csv_text_collection.add(
                ids=[f"text_{idx}"],  # Adding unique ids for each text chunk
                embeddings=[text_embedding],
                metadatas=[{"row_index": idx}],
                documents=[text]
            )

        # Generate and store embeddings for context
        if context:  # Check if context is not empty
            context_embedding = hf_embeddings.embed_documents([context])[0]  # Get the first (and only) embedding from the list
            csv_context_collection.add(
                ids=[f"context_{idx}"],  # Adding unique ids for each context chunk
                embeddings=[context_embedding],
                metadatas=[{"row_index": idx}],
                documents=[context]
            )


In [17]:
csv_filename = 'Barney.csv'
csv_path = os.path.join(current_directory, 'DATA', csv_filename)
process_csv_file(csv_path)

In [21]:
import pickle

In [22]:
# Save collections with embedings
def save_all_collections(collections, filename):
    all_data = {}

    for name, collection in collections.items():
        # Fetch data from the collection
        data = {
            "ids": collection.get()["ids"],
            "embeddings": collection.get()["embeddings"],
            "metadatas": collection.get()["metadatas"],
            "documents": collection.get()["documents"],
        }
        all_data[name] = data

    # Save all data to a single file
    with open(os.path.join(save_directory, filename), 'wb') as file:
        pickle.dump(all_data, file)

In [23]:
# Get collections
collections = {
    "text_embeddings": client.get_collection(name="text_embeddings"),
    "csv_text_embeddings": client.get_collection(name="csv_text_embeddings"),
    "csv_context_embeddings": client.get_collection(name="csv_context_embeddings"),
}

In [24]:
# Save all collections to one file
save_all_collections(collections, 'all_collections.pkl')